# Problem 3
## Problem 11 Chapter 6

We will now try to predict per capita crime rate in the Boston data
set.<br>

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import scatter_matrix
from IPython.core.interactiveshell import InteractiveShell
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures


from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy.linalg import inv
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

InteractiveShell.ast_node_interactivity = "all"
#%matplotlib notebook

In [2]:
# Load the data
filepath = '../../datasets/ISLR/data/csv/'
filename = 'Boston.csv'
df = pd.read_csv(filepath + filename)

In [3]:
# Get a numpy array and scale the data
values = df.values
scaler = StandardScaler()
scaled_data = scaler.fit_transform(values)

In [4]:
#print(df.columns)
#df.isnull().sum()
#print(scaled_data.shape)
X_train, X_test, y_train, y_test = train_test_split(scaled_data[:,1:], scaled_data[:,0])

### Part A
Try out some of the regression methods explored in this chapter,
such as best subset selection, the lasso, ridge regression, and
PCR. Present and discuss results for the approaches that you
consider.

In [5]:
# Keep track of the different model scores
scores = {}

#### $L^1$ Regularization (Lasso)

In [6]:
# Train using gridsearch cv
alphas = np.logspace(-3, 1, num=10)
parameters = {'alpha':alphas}
lasso = Lasso()
model = GridSearchCV(lasso, parameters, cv=5, return_train_score=True)
_ = model.fit(scaled_data[:,1:], scaled_data[:,0])

In [7]:
y_hat = model.predict(X_test)
score = mean_squared_error(y_test, y_hat)
scores['Lasso'] = score
print(score)

1.4568842806962454


In [8]:
# I want to see the weights chosen by Lasso
keys = df.columns.values.tolist()
_ = keys.pop(0)
values = model.best_estimator_.coef_
weights = dict(zip(keys, values))
for k, v in weights.items():
    print('{}:   {}'.format(k, v))

zn:   0.0
nox:   0.0
ptratio:   -0.0
black:   -0.06123326215880774
medv:   -0.051921576446071545
indus:   0.0
dis:   -0.007571552731945718
rad:   0.46364425449909236
age:   0.0
lstat:   0.1046339374198605
chas:   -0.0
tax:   0.0
rm:   -0.0


In [9]:
# Try dropping zero columns and adding polynomial versions of non-zero columns
df_copy = df.copy()
#drops = list(weights.keys()[weights.values() == 0])
keeps = []
for key in weights.items():
    if key[1] != 0:
        keeps.append(key)

keeps = [x[0] for x in keeps]
df_copy = df_copy[keeps]

# Get a numpy array and scale the data
values_copy = df_copy.values
scaler_copy = StandardScaler()
scaled_data_copy = scaler.fit_transform(values_copy)

# Make PolynomialFeatures object
poly = PolynomialFeatures(2)
poly_data = poly.fit_transform(scaled_data_copy)

In [10]:
# Make new Lasso and fit again
# Train using gridsearch cv
alphas = np.logspace(-3, 1, num=10)
parameters = {'alpha':alphas}
lasso = Lasso()
model = GridSearchCV(lasso, parameters, cv=5, return_train_score=True)
X_train, X_test, y_train, y_test = train_test_split(poly_data, scaled_data[:,0])
_ = model.fit(X_train, y_train)

In [11]:
y_hat = model.predict(X_test)
score = mean_squared_error(y_test, y_hat)
scores['Lasso- Poly'] = score
print(score)

0.5126264698921603


#### $L^2$ Regularization (Ridge)

In [12]:
alphas = np.logspace(-3, 1, num=10)
parameters = {'alpha':alphas}
ridge = Ridge()
model = GridSearchCV(ridge, parameters, cv=5, return_train_score=True)
_ = model.fit(scaled_data[:,1:], scaled_data[:,0])

In [13]:
y_hat = model.predict(X_test)
score = mean_squared_error(y_test, y_hat)
scores['Ridge'] = score
print(score)

ValueError: shapes (127,21) and (13,) not aligned: 21 (dim 1) != 13 (dim 0)

In [ ]:
# I want to see the weights chosen by Ridge
keys = df.columns.values.tolist()
_ = keys.pop(0)
values = model.best_estimator_.coef_
weights = dict(zip(keys, values))
for k, v in weights.items():
    print('{}:   {}'.format(k, v))

In [ ]:
# Make new Ridge and fit again using polydata
# Train using gridsearch cv
alphas = np.logspace(-3, 1, num=10)
parameters = {'alpha':alphas}
ridge = Ridge()
model = GridSearchCV(ridge, parameters, cv=5, return_train_score=True)
X_train, X_test, y_train, y_test = train_test_split(poly_data, scaled_data[:,0])
_ = model.fit(X_train, y_train)

In [ ]:
y_hat = model.predict(X_test)
score = mean_squared_error(y_test, y_hat)
scores['Ridge- Poly'] = score
print(score)

In [ ]:
for k, v in scores.items():
    print('{}:   {}'.format(k, v))

### Part B
Propose a model (or set of models) that seem to perform well on
this data set, and justify your answer. Make sure that you are
evaluating model performance using validation set error, crossvalidation,
or some other reasonable alternative, as opposed to
using training error.

### Part C
Does your chosen model involve all of the features in the data
set? Why or why not?